# テーブルとデータの準備

In [1]:
!pip install intersystems-irispython
!pip install pillow
!pip install transformers==4.49.0
!pip install timm
!pip install torch
!pip install torchvision
!pip install sentencepiece
!pip install protobuf



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import iris
import io
import requests
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer
import csv
import torch.nn.functional as F
import os

# ---------------------------------------------
# 接続情報
# ---------------------------------------------
IRIS_HOST = "localhost"
IRIS_PORT = 51782
IRIS_NAMESPACE = "USER"
IRIS_USER = "_SYSTEM"
IRIS_PASSWORD = "SYS"

IRIS_CONN_STR = f"{IRIS_HOST}:{IRIS_PORT}/{IRIS_NAMESPACE}"

In [16]:
# ---------------------------------------------
# 魚名ベクトル テーブルを作成する
# ---------------------------------------------
def create_tables():
    """
    IRIS に FishSearch.Fish および FishSearch.FishImage テーブルを作成する。
    すでに存在する場合はスキップされる。
    """
    conn = iris.connect(IRIS_CONN_STR, IRIS_USER, IRIS_PASSWORD)
    cur = conn.cursor()

    try:
        # Fish テーブル
        cur.execute("""
            CREATE TABLE IF NOT EXISTS FishSearch.Fish (
                Name VARCHAR(100),
                Features VECTOR(DOUBLE, 512)
            )
        """)

        # FishImage テーブル
        cur.execute("""
            CREATE TABLE IF NOT EXISTS FishSearch.FishImage (
                Name VARCHAR(100),
                Features VECTOR(DOUBLE, 512)
            )
        """)

        conn.commit()
        print("テーブル作成（または既存確認）完了")
    
    finally:
        cur.close()
        conn.close()

In [11]:

# ----------------------------------------
# IRIS に埋め込みを登録する関数
# ----------------------------------------
def insert_embedding(
                     name,
                     embedding_tensor,
                     modality):
    """
    embedding_tensor : shape = [512] の 1D Tensor
    """
    # ① Tensor → list[float] → カンマ区切り文字列
    float_list = embedding_tensor.cpu().tolist()        # 長さ 512
    emb_str = ",".join(map(str, float_list))            # "0.123,0.456, …"

    # ② IRIS へ接続して INSERT
    conn = iris.connect(IRIS_CONN_STR, IRIS_USER, IRIS_PASSWORD)
    cur  = conn.cursor()

    table = 'FishSearch.Fish' if modality == 1 else 'FishSearch.FishImage'
    
    try:
        sql = "INSERT INTO " + table + " (Name, Features) VALUES (?, TO_VECTOR(?, DOUBLE, 512))"
        cur.execute(sql, [name, emb_str])
        conn.commit()
        print("INSERT 完了")
    finally:
        cur.close()
        conn.close()

In [17]:
# 必要なテーブルを作成する
create_tables()

テーブル作成（または既存確認）完了


In [12]:

# LINEヤフーさんが提供している日本語対応のCLIPモデルを利用
HF_MODEL_PATH = 'line-corporation/clip-japanese-base'
device = "cuda" if torch.cuda.is_available() else "cpu"

# トークナイザー(テキストをモデル入力用のトークンに変換する役割)の読み込み
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
# 画像前処理（リサイズ・正規化など） を行うプロセッサーを読み込み
processor = AutoImageProcessor.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
# CLIP モデル本体を読み込み、指定したデバイス（CPU/GPU）に配置
model = AutoModel.from_pretrained(HF_MODEL_PATH, trust_remote_code=True).to(device)


In [ ]:
### CSVを読み込み、ベクトルを取得し、IRISに登録
# --- CSV ファイルの読み込み ---
csv_path = "../datasets/fishname.csv"

with open(csv_path, "r", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # ヘッダー行をスキップ
    for row in reader:
        if not row or not row[1].strip():
            continue  # 空行をスキップ

        label = row[1].strip()
        text = tokenizer(label).to(device)

        # --- 埋め込みベクトルの生成 ---
        with torch.no_grad():
            text_features = model.get_text_features(**text)  # shape: [1, 512]

        #　--- L2ノルムで正規化 ---
        text_features = F.normalize(text_features, p=2, dim=1)
        
        # ベクトルを IRIS に登録
        modality = 1
        insert_embedding(label, text_features[0], modality)


INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了


In [19]:
###　画像もデータベースに登録


# 対象フォルダのリストを定義（必要なだけ追加可能）
image_dirs = [
    "../datasets/SELECTED/",
    "../datasets/Wikipedia/image/"
]

# 各フォルダを順に処理
for image_dir in image_dirs:
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

    for filename in image_files:
        image_path = os.path.join(image_dir, filename)

        try:
            image = Image.open(image_path).convert("RGB")
            inputs = processor(image, return_tensors="pt").to(device)

            with torch.no_grad():
                image_features = model.get_image_features(**inputs)

            #　--- L2ノルムで正規化 ---
            image_features = F.normalize(image_features, p=2, dim=1)
            
            modality = 2  # IMAGE
            insert_embedding(filename, image_features[0], modality)

        except Exception as e:
            print(f"画像処理失敗: {filename} - {e}")

INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了
INSERT 完了


In [ ]:
# TODO 件数の表示
# TODO 最初の数件の表示



# IRIS へ接続
conn = iris.connect(IRIS_CONN_STR, IRIS_USER, IRIS_PASSWORD)
cur  = conn.cursor()

table = '' 

# try:
#     sql = "SELECT Name, Features FROM  FishSearch.Fish"
#     cur.execute(sql, [name, emb_str])
    
#     print("INSERT 完了")
# finally:
#     cur.close()
#     conn.close()


try:
    sql = "SELECT Name, Features FROM  FishSearch.Fish"

    with conn.cursor() as cursor:
        cursor.execute(sql)
        results = cursor.fetchall()
    for name, features in results[:10]:  # 最初の10行を処理
        preview = features[:75] + "..."  # features 一部を表示
        print(f"{name}: {preview}")

except Exception as e:
    print(f"Error: {str(e)}")

マアジ: .0030519193969666957856,.019338253885507583618,-.026509894058108329772,-.00...
カツオ: .013707441277801990509,.020661199465394020081,-.044357202947139739991,.0512...
サクラマス: -.032789152115583419799,.020647652447223663331,-.048449773341417312622,.015...
アユ: .014848685823380947113,.0040654172189533710479,-.033739592880010604858,-.03...
コイ: .047572702169418334961,-.0045570936053991317749,-.012862112373113632202,.03...
ゼブラフィッシュ: .062812961637973785401,-.0027012170758098363876,.012029914185404777526,.007...
アカマンボウ: .011026332154870033264,-.010659807361662387847,-.05038928985595703125,.0348...
ヒラスズキ: .013157647103071212768,.027062788605690002441,.020210636779665946961,-.0493...
タチウオ: .025754258036613464356,.049810905009508132934,-.016243657097220420837,-.011...
ギンダラ: .052557989954948425292,.012833195738494396209,.011079670861363410949,-.0128...
